In [3]:
import pandas as pd
import numpy as np
import re

from io import StringIO

from sklearn.externals import joblib
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.datasets import make_regression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer


In [99]:
###читаем даатасет c фичами фильмов
df_f = pd.read_csv('./movie_titles.csv', names=['MovieID', 'Year', 'Title'])
df_f = df_f.set_index('MovieID')

###читаем даатасет с инфой о пользователях
f = open('./combined_data_1.txt')
file = f.read()

movieID_u = list(map(lambda x: int(x[:-1]), re.findall(r'\d+\:', file)))
df_u = re.split(r'\d+\:', file)
df_u = df_u[1:]    

df = []
for df_i, movieID_i in zip(df_u, movieID_u):
    sub_df = pd.read_csv(StringIO(df_i), names=['UserID', 'Score', 'Date'])
    sub_df['MovieID'] = movieID_i
    df.append(sub_df)


df = pd.concat(df)
df_f = df_f.drop(df_f[df_f.index>df.MovieID.unique().shape[0]].index)
#print(df_f)
#print(df)


In [100]:
u_size = df.UserID.unique().shape[0]
m_size = df.MovieID.unique().shape[0]
#y_size = df_f.Year.unique().shape[0]
#s_size = df.Date.unique().shape[0]

In [103]:
df.sort_values(by=['UserID', 'MovieID'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [120]:
### разреженная матрица
sm = lil_matrix((df.shape[0], u_size + m_size))
#print(sm.shape)

In [121]:
j=0
temp = df.UserID.iat[0]

for i in range(df.shape[0]):
    
    if(temp<df.UserID.iat[i]):
        j+=1
        temp = df.UserID.iat[i]
        
    sm[i, j] = 1
    sm[i, u_size + df.MovieID.iat[i]-1] = 1
    
    if i % 50000 == 0:
        print("\r Progress: {}/{} ({}%)".format(i, df.shape[0], int(((i+1) / df.shape[0]) * 100)), end="")
    #print(i, j, sm[i,j])
    #print(i, j1, sm[i,j1])
print(sm[0,0])

 Progress: 24050000/24053764 (99%)1.0


In [122]:
sm = csr_matrix(sm)
joblib.dump(sm, 'sparse_df_1.bin')

['sparse_df_1.bin']

In [123]:
sm = joblib.load('sparse_df_1.bin')

In [127]:
print(sm[453, :])

  (0, 6)	1.0
  (0, 473670)	1.0


По лемме 1 нелинейную часть можно представить в виде: $\sum_{i=1}^n \sum_{j=i+1}^n \langle {v}_i, {v}_f\rangle x_ix_j = \frac{1}{2}\sum_{f=1}^k \left( \left(\sum_{i=1}^n v_{i,f}x_i \right)^2 - \sum_{i=1}^n v^2_{i,f}x^2_i \right)$
<br>
Производная $\frac{dy}{dv}$ нелинейной части бдует следующей:
<br>
(1) = $\left( \left( \sum_{i=1}^n v_{i,f}x_i \right)^2 \right)' _{v_if} = 2\sum_{i=1}^n \left(v_{i,f}x_i \sum_{k=1}^n x_k\right)$ или в матричном виде $\sum_{f=1}^k2 (X\mathbb{1})^TXV_f$
<br>
(2) = $ \left( \sum_{i=1}^n v_{i,f}^2x_i^2 \right)' _{v_if} = 2\sum_{i=1}^n v_{i,f}x_i^2 $ или в матричном виде $\sum_{f=1}^k2 X^TXV_f$
<br>
<br>
Тогда
<br>
$\frac{dy}{dv_f} = \frac{1}{2}\sum_{f=1}^k \left( (1) - (2) \right)$ или в матричном виде $\sum_{f=1}^k(X\mathbb{1}-X)^TXV_f$
<br>
<br>
И тогда $\frac{dy}{dv}$ в матричном виде $(X\mathbb{1}-X)^TXV\mathbb{1}$

In [ ]:
def RMSE(x, y):
    return np.sqrt(np.sum(np.power(y - x, 2)) / y.shape[0])


def narrTheta(lr, iter_num):
    return lr / np.sqrt(iter_num)


def sgdFFM(X, y, max_iter=1000, step=1e-3, batch_size=256, k=8):
    w0 = 0
    iter_num = 1
    N = X.shape[0]
    #print(X, batch_size)
    np.random.seed(42)
    w1 = np.random.normal(size=X.shape[1])

    while iter_num <= max_iter:
        random_batch = np.random.choice(N, batch_size)
        new_y = predict(X[random_batch, :], w0, w1)
        # print(X[random_batch].shape)

        dy = 2 * (new_y - y[random_batch]) / N 
        # print(iter_num, dy)
        w0 -= dy.mean() * step
        w1 -= X[random_batch, :].T @ dy * step#narrTheta(step, iter_num)
        iter_num += 1
        if (iter_num % 5 == 0):
            print(np.linalg.norm(old_y - new_y))

        old_y = new_y
    return w0, w1



def predict(X, w0, w1):
    return w0 + X @ w1


In [ ]:
X, y, coefs = make_regression(n_samples=1000000, n_features=8, n_targets=1, n_informative=4, coef=True)

folds_index = 5
fold_size = round(X.shape[0] / folds_index)
RMSE_test = []
RMSE_train = []

for i in range(folds_index):
    test = X[i * fold_size:(i + 1) * fold_size]
    testT = y[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = X[(i + 1) * fold_size:, :]
        trainT = y[(i + 1) * fold_size:]
    else:
        train = X[:i * fold_size, :]
        trainT = y[:i * fold_size]
        if i != 4:
            train = np.concatenate((train, X[(i + 1) * fold_size:, :]))
            trainT = np.concatenate([trainT, y[(i + 1) * fold_size:]])
    
    Target = trainT
    Features = train
    w0, w1 = sgdFFM(Features, Target, 1e5, 1e-3, 256)

    train_pred = predict(Features, w0, w1)

    # RMSE_train.append(RMSE(train_pred, Target))
    TargetT = testT
    FeaturesT = test
    test_pred = predict(FeaturesT, w0, w1)
    # RMSE_test.append(RMSE(test_pred, Target))

# print('Fold {}'.format(i + 1), '\nОбучающая: \n', train_pred - Target, '\nТестовая: \n', test_pred - TargetT, '\n', '\n')


54.72861590556448
57.90865281739233
58.43664882701668
58.279073237685566
59.34030803870171
56.17991071292963
58.21575720714855
52.31278885331447
56.873904281957856
52.535279824443
55.41018978638869
57.46179005772193
55.911297598176894
55.90925956273409
59.336899188800494
58.26929341718697
56.63245335487067
54.98195205796841
54.16004451589719
56.84657225326217
55.58835058329153
56.64362479303038
56.54428539654698
55.64680500953413
58.70775627329097
53.585938765594356
55.97356118245351
55.822820468240614
60.82041692602215
55.93944617168251
57.340359494817584
59.306144442343246
58.66396884497934
55.448914371944156
59.31654790332178
58.089354738214034
55.34486919915772
55.558163458787625
52.51105132885528
55.37436619340549
59.31368231051367
55.81167125438165
57.92220051414305
54.75575498130886
56.692836996802434
55.82303971604622
55.490234118502165
55.51744297340606
56.09636626188135
57.352301179013324
62.26624381334495
51.98980581555723
56.821241072853304
57.07869504109618
55.546033311168

54.05305331602354
60.60706861986455
56.465136789965136
57.60977501970567
55.46357892703342
56.38195589313018
58.93345225575806
59.49733933095648
54.02017898910764
53.31672243406518
57.779417336893
50.799449756807775
60.05780119845646
55.177701579373796
55.63712418061
56.455142112786625
54.11947501193602
57.15826013422578
52.87186811471435
57.918931704146786
56.60699389075747
56.615150982659735
54.09986757205016
55.95453543531692
57.599208561236004
58.47527065511407
56.68962517488373
55.024556273521306
56.1241306826562
57.32557786660916
57.044913233707376
55.63995142798695
59.67138736587021
55.48171144586227
58.29408440833
54.80651660122598
56.17371118945618
54.26088762735282
53.81932737087038
54.578034570431
60.78828174514251
57.182598118826554
52.953809912528364
59.22402607733982
53.02145789242485
55.300598026870155
56.46768129491102
55.93642732094677
51.46545754256884
57.00028816315389
53.917908570407356
56.41848287025679
55.049010466515305
58.911194978456386
59.60181467121648
53.734

60.970029036247304
57.836100047242695
60.54929316075926
60.12597631011088
51.857574372361356
56.82506025113635
59.2993966891113
57.27913826242639
56.718507941424896
55.70286054826855
55.255090157285245
53.89830145014325
57.63742467987547
56.771529837491634
60.175080109608324
57.89255766955674
59.74657951653761
55.32049155957336
56.96602165682856
55.17751988081238
60.40431187495904
52.71943098366869
59.1641632446185
57.57041617937948
55.48968964564282
60.97628764493507
56.54318640647886
55.1976497457051
57.809443402408924
60.33597063146213
52.05082173451315
56.062567058663525
53.87114838072002
59.289129088766146
61.888499188879045
60.23626013721843
57.09911839094612
58.833463898211775
56.07036735954757
58.830871489264894
57.408772848893456
51.600941275040476
60.482753430092785
50.886749978915816
56.224733225485814
58.874732317964046
54.8194658170984
60.55778191802255
57.45469519070072
64.13667946447418
56.621371218292
55.32874473098246
59.69859815705575
57.598696372432684
60.86174274333

55.836560993308424
55.63984328419716
57.849656603225725
58.98009595605959
60.65112066036885
57.30365914160578
55.4777417076529
55.40734778716722
56.48861254351131
58.32792303331538
55.64346106989278
62.01308303173499
55.38141223813529
54.33997815914548
57.76698106257758
58.33156592951167
58.239422449509775
57.26799883063723
58.16031584601756
56.006931439277444
58.86965891293282
59.67279824121978
56.33157258887163
58.00467615855569
57.763522565955554
60.39577423444996
59.82850089097217
62.648519772617526
59.252644819236416
60.838197183986686
58.108939298156216
58.89227821565984
52.41433911265937
62.03764304201867
58.026688131589154
58.96309700586088
57.89862111183629
57.684055234085285
60.54658038413807
61.05925267633504
57.162530049506884
56.92273857994653
58.486321366179496
57.84133662393554
55.152166331136804
53.64140075050256
60.68552757843416
57.68478532743601
56.10296499972421
56.778493249016904
57.40077450944717
55.57821055600289
55.64969499742844
57.80458155023558
54.12210930731

60.681205321197645
58.173698639450436
60.36737600454877
62.987295724985216
60.04826038426714
66.68552676599681
60.88450446277665
59.55684468623551
62.31761454964321
60.633933297387905
55.193039335203984
56.94888160449365
55.75178622206509
58.540037611794396
59.03160950945176
60.08381641631691
58.95782432871466
60.09692096423628
56.07490324688741
61.19240041516542
59.98614310945454
62.27489053411743
57.77628814426478
61.54923813449321
63.03950186784915
57.22045864019314
56.46701746868731
57.29277572154294
58.003967821227796
62.93320557364027
58.98161900056045
60.25018344421326
57.473039123161904
60.46315886287153
59.98848725813882
60.46885461779628
57.839831222453896
59.74737596681473
57.86392450604861
57.055916645040455
59.29593484057628
60.15166148142529
59.831021588920585
58.61050963871904
64.61546706825669
59.33855182550192
65.5043119542118
59.617318196531144
58.15126869992581
61.320861191781134
60.84792661230674
58.39841461443056
61.18704273402245
57.65699176319817
60.3084247959100

...

110.28047200670079
106.1449282337394
107.76586544690755
107.8026381781114
101.78968283029306
109.0132401617879
104.31624292030426
108.3034624005875
111.64436807375377
105.39682296147016
105.05947091427679
99.82500311297751
116.45404700951082
108.59509041490897
113.13829809383284
106.85602058542776
104.95547963264765
105.17036813773187
105.04645362886566
108.77568619122442
104.96281716367507
105.32195047601303
107.73446463852733
98.60804094529861
111.41017232004586
108.62056857971241
102.50552032834103
100.99163843325194
103.89542189003521
104.85883475206641
108.19583572291678
107.247069322778
110.64073697911117
109.50172286115492
103.73409575848615
101.71776655816464
107.54216703009057
109.7665113404776
110.77547252806748
105.29173688272334
104.44221574829005
106.14902729758174
105.82162266550093
104.94914428978983
112.31397829746419
104.14271243667905
112.89665330958103
103.08258974988394
112.89564576312043
103.34443096568646
110.58856647522778
112.07825748669576
114.50648399441489
11

111.49510207406223
115.16499091878778
106.41686991494431
122.48203516280981
109.83943019622292
105.47738951367256
114.20406821336375
111.92011947803853
120.4385395696719
123.23969822891134
114.40403859489868
120.91686643348356
117.12823899432041
112.20161292653061
113.28401835913826
112.5359100421693
115.47323215387821
112.54924517572078
107.25314943587075
111.01572295092849
103.96085994071751
105.70250824403047
111.41296729075522
104.83170460031232
110.58904356290898
116.31490339818409
106.96706942001072
112.14989796842133
109.13186337110973
110.09308935928362
104.34751524800812
113.10958430307159
110.19679227719958
113.39901423823873
104.99085506058124
108.7242201021702
109.0242014245679
104.56656839562444
109.86859945483778
113.12621154045429
109.45859380519492
115.53857165908647
111.41514839511277
113.44563491329947
113.20518962862918
102.32821344528693
107.78547906709524
111.30176882254167
118.4826121759359
113.31327165121355
115.10128755950647
120.58783339663108
103.3347866988201

108.59084752154693
112.3952589810693
115.13645747920854
112.80907079768416
119.55009383863191
114.88582736689222
122.44246987559785
110.06693662178338
102.2914820391106
114.51784458146328
109.60726975204612
118.01410092034777
111.97306438349311
118.96318909348729
114.3999961783354
115.50763132593868
121.94250319199571
111.21637968284553
117.57707556572463
113.12894950392479
116.57769006148278
110.99203105492896
112.6830905004268
119.17339986022452
118.76948399262713
112.80150738527021
117.91927880992604
112.89439187204664
116.10222346340825
107.67273930829245
108.22626502536312
124.65180428651173
120.6481870887003
114.3443685113096
117.54453359049174
113.61010167964197
115.82156492359658
114.17008341576883
113.53810406273645
115.59429628257291
111.53704453198132
119.87398826342918
120.69801899937652
116.98354203736653
106.71270237255261
109.63570973387218
120.20983998124727
113.27524403688017
115.94933739499284
111.43238039450874
117.54631166440663
106.73917425179391
109.6997229671392


117.40023287198017
118.78363629156898
122.16813938932674
131.0843642532391
118.22922579374801
118.00824888354902
106.53221809347242
111.86167860584594
120.33840025441407
104.74088621594045
120.5292445447638
116.87442684871145
119.21352838125357
112.51866966484164
111.27814456123762
115.71432846539669
121.8317761557812
118.53059595662737
124.25029486165262
126.21771355039269
125.92516372118735
124.18913878615473
115.43181488884422
128.01480449072744
114.38204772516056
120.26203451946043
120.62012623104802
130.20905583745642
111.98928244612725
123.72246397166514
118.65337086672707
112.88044164400587
124.43425916944068
115.49772127667694
117.37746450237458
118.3886205535161
122.44841765778115
128.07301526639165
113.5629664657304
127.18981589229556
115.52746782745274
122.22712637415847
122.42448564962099
118.15289729313982
116.27721446371889
116.44215978900192
110.2515441145931
121.69257050194014
118.59403816798985
121.4407486865607
120.91034625837247
132.09296854514344
122.81345073171754


123.6236856199533
119.89319121223014
126.00230760098589
116.57661677459146
121.4892486553951
122.36912583836
126.18947402813814
123.2974429989139
124.14474540112485
127.46681430519
125.20756058358229
130.13386285597645
129.27645822661472
128.87289458089475
128.96682946813596
126.0998837867186
122.88365433431633
122.67275692775772
120.87007913399384
124.01465254855889
119.06422710016889
124.05901639081286
119.28522503242468
121.18270142900143
119.1010022035666
122.63244808669636
133.910549891671
128.77828891752492
129.47069371427028
134.34114745029714
120.48316327520774
125.16567397864077
123.10097647910625
121.8595899206844
124.77168532690457
125.83421684161797
126.49471511883603
119.35378695651652
129.03677063042215
124.08443296746694
117.74870800818951
117.17711043607424
131.76061330235203
123.2347540148657
120.37658830372837
129.40427769914206
122.4413258592497
122.28665437425934
121.8362676186244
119.2034506258262
119.23743255147498
120.87941346981339
126.27772123850062
118.7587586